In [51]:
import datalab.storage as storage
import pandas as pd
from io import BytesIO
import numpy as np
import random

In [52]:
bucket = storage.Bucket('cs221-flight-data')
flights_data = bucket.item('model-train.csv')
uri = flights_data.uri
%gcs read --object $uri --variable data
bayes_data_train = pd.read_csv(BytesIO(data))
flights_data = bucket.item('model-test.csv')
uri = flights_data.uri
%gcs read --object $uri --variable data
bayes_data_test = pd.read_csv(BytesIO(data))
bayes_data = pd.concat(bayes_data_train, bayes_data_test)

In [53]:
# CONSTANTS
NUM_TIME_DELAY_BUCKETS = 5
NUM_AIRPLANE_DELAY_BUCKETS = 5
la_place_constant = 1

# map the variables to their parents
variables_to_parents = {}
variables_to_parents['MONTH'] = []
variables_to_parents['DAY_OF_WEEK'] = []
variables_to_parents['SCHEDULED_DEPARTURE'] = []
variables_to_parents['time_delay'] = ['MONTH', 'DAY_OF_WEEK', 'SCHEDULED_DEPARTURE']
variables_to_parents['AIRLINE'] = []
variables_to_parents['DESTINATION_AIRPORT'] = []
# variables_to_parents['YEAR'] = [] # year is the year of the airplane make NOT the year that the flight departs
# variables_to_parents['MODEL'] = []
# variables_to_parents['airplane_delay'] = ['YEAR', 'MODEL']
# variables_to_parents['ARRIVAL_DELAY'] = ['time_delay', 'AIRLINE', 'DESTINATION_AIRPORT', 'airplane_delay']
variables_to_parents['ARRIVAL_DELAY'] = ['time_delay', 'AIRLINE', 'DESTINATION_AIRPORT']


In [54]:
# initialize the counts
variable_to_counts = {}
# these variables don't have parent variables
for var in variables_to_parents:
    if len(variables_to_parents[var]) == 0:
        variable_to_counts[var] = {key:la_place_constant for key in np.unique(bayes_data[var].astype(str).unique())}

# variable_to_counts['airplane_delay'] = {(airplane_delay, year, model):random.randint(1,10)
                                 #      for airplane_delay in range(NUM_AIRPLANE_DELAY_BUCKETS)
                                  #     for year in bayes_data['YEAR'].astype(str).unique()
                                   #    for model in bayes_data['MODEL'].astype(str).unique()}

In [55]:
# add the variable with parent variables
# make sure that the order of the tuple is the same order as the parent list
variable_to_counts['time_delay'] = {(time_delay, month, day_of_week, sch_departure):random.randint(1,3)
                                    for time_delay in range(NUM_TIME_DELAY_BUCKETS)
                                    for month in bayes_data['MONTH'].astype(str).unique()
                                    for day_of_week in bayes_data['DAY_OF_WEEK'].astype(str).unique()
                                    for sch_departure in bayes_data['SCHEDULED_DEPARTURE'].astype(str).unique()}

In [56]:
# add the variable with parent variables
# make sure that the order of the tuple is the same order as the parent list
variable_to_counts['ARRIVAL_DELAY'] = {(arrival_delay, time_delay, airline, dest_airport):la_place_constant
                                      for arrival_delay in bayes_data['ARRIVAL_DELAY'].astype(str).unique()
                                      for time_delay in range(NUM_TIME_DELAY_BUCKETS)
                                      for airline in bayes_data['AIRLINE'].astype(str).unique()
                                      for dest_airport in bayes_data['DESTINATION_AIRPORT'].astype(str).unique()}
                                      # for airplane_delay in range(NUM_AIRPLANE_DELAY_BUCKETS)}

In [ ]:
flights_data = bucket.item('model-train.csv')
uri = flights_data.uri
%gcs read --object $uri --variable data
bayes_data = pd.read_csv(BytesIO(data))

In [57]:
# count to get the probabilities for the quantities that we DO know
for index, row in bayes_data.iterrows():
    for (variable, parent_list) in variables_to_parents.items():
        if variable != "time_delay":
          # need to make tuple of value with their parent(conditional) values
          value = str(row[variable])
          if parent_list != []:
            value = [value]
            for parent in parent_list:
              if parent != "time_delay":
                value.append(str(row[parent]))
              elif parent == "time_delay":
                value.append(random.randint(0, NUM_TIME_DELAY_BUCKETS-1))
            value = tuple(value)
          # increase the count
          variable_to_counts[variable][value] += 1

In [58]:
# once the counts are initialized, need to normalize the probabilities 
def normalize(variables_to_parents, variable_to_counts):
  variable_to_probability = {}
  for variable in variable_to_counts:
      variable_to_probability[variable] = {}
      if len(variables_to_parents[variable]) > 0:
          # then we need to do some fun tuple stuff because the conditionals are what is constant
          conditional_to_count = {}
          for value in variable_to_counts[variable]:
              conditional = value[1:]
              conditional_to_count[conditional] = conditional_to_count.get(conditional, 0) + variable_to_counts[variable][value]
          # now actually get the percentage
          for value in variable_to_counts[variable]:
              conditional = value[1:]
              variable_to_probability[variable][value] = float(variable_to_counts[variable][value])/conditional_to_count[conditional]
      else:
          # there are no parents
          for value in variable_to_counts[variable]:
              # add all of the counts for all of the values for the variable
              total = sum(variable_to_counts[variable].values())
              variable_to_probability[variable][value] = float(variable_to_counts[variable][value])/total
  return variable_to_probability

In [59]:
variable_to_probability = normalize(variables_to_parents, variable_to_counts)

In [60]:
# E-step
def e_step(data, var_to_prob, variables_to_parents):
  assignment_to_curr_prob = {}
  known_var_to_total = {}
  variables_sorted = sorted(variables_to_parents.keys())
  time_delay_var_index = variables_sorted.index("time_delay")
  for index, row in data.iterrows():
    # consider all possible values of the hidden variable
    for time_delay_val in range(NUM_TIME_DELAY_BUCKETS):
      # determine joint probability
      prob = 1
      assignment = []
      # sort the (variable, parent_list) tuples by variable name, this ensures that assigments are always in the same order
      for (variable, parent_list) in sorted(variables_to_parents.items(), key=lambda tup: tup[0]):
        if variable != "time_delay":
          value = str(row[variable])
        elif variable == 'time_delay':
          value = time_delay_val
        assignment.append(value)
        if parent_list != []:
          value = [value]
          for parent in parent_list:
              if parent != "time_delay":
                value.append(str(row[parent]))
              elif parent == "time_delay":
                value.append(time_delay_val)
          value = tuple(value)
        prob *= var_to_prob[variable][value]
      # add the joint probability to the map of assigments
      known_vars = assignment[:time_delay_var_index]
      known_vars.extend(assignment[time_delay_var_index+1:])
      known_vars = tuple(known_vars)
      known_var_to_total[known_vars] = known_var_to_total.get(known_vars, 0) + prob
      assignment = tuple(assignment)
      assignment_to_curr_prob[assignment] = prob
  # now I need to normalize the probabilities
  assigment_to_norm_prob = {}
  for (assignment, prob) in assignment_to_curr_prob.items():
    known_vars = list(assignment[:time_delay_var_index])
    known_vars.extend(assignment[time_delay_var_index+1:])
    known_vars = tuple(known_vars)
    total = known_var_to_total[known_vars] 
    assigment_to_norm_prob[assignment] = float(prob) / total
  return assigment_to_norm_prob

In [61]:
def initialize_counts(variables_to_parents, smoothing_constant, bayes_data):
  variable_to_counts = {}
  for var in variables_to_parents:
    if len(variables_to_parents[var]) == 0:
        variable_to_counts[var] = {key:smoothing_constant for key in np.unique(bayes_data[var].astype(str).unique())}
  variable_to_counts['time_delay'] = {(time_delay, month, day_of_week, sch_departure):smoothing_constant
                                    for time_delay in range(NUM_TIME_DELAY_BUCKETS)
                                    for month in bayes_data['MONTH'].astype(str).unique()
                                    for day_of_week in bayes_data['DAY_OF_WEEK'].astype(str).unique()
                                    for sch_departure in bayes_data['SCHEDULED_DEPARTURE'].astype(str).unique()}
  variable_to_counts['ARRIVAL_DELAY'] = {(arrival_delay, time_delay, airline, dest_airport):smoothing_constant
                                      for arrival_delay in bayes_data['ARRIVAL_DELAY'].astype(str).unique()
                                      for time_delay in range(NUM_TIME_DELAY_BUCKETS)
                                      for airline in bayes_data['AIRLINE'].astype(str).unique()
                                      for dest_airport in bayes_data['DESTINATION_AIRPORT'].astype(str).unique()}
                                      # for airplane_delay in range(NUM_AIRPLANE_DELAY_BUCKETS)}
  return variable_to_counts

In [62]:
def m_step(assignment_to_prob, variables_to_parents, smoothing_constant, bayes_data):
  variables_sorted = sorted(variables_to_parents.keys())
  # need way of translating assignment index to variable value
  variable_to_assigment_index = {variables_sorted[i]:i for i in range(len(variables_sorted))}
  # initialize the new counts map
  variable_to_counts = initialize_counts(variables_to_parents,smoothing_constant, bayes_data)
  # for every assignment, add the value of its variables to the counts
  for (assignment, weight) in assignment_to_prob.items():
    for (variable, parent_list) in variables_to_parents.items():
      value = assignment[variable_to_assigment_index[variable]]
      if parent_list != []:
        value = [value]
        for parent in parent_list:
           value.append(assignment[variable_to_assigment_index[parent]])
        value = tuple(value)
      variable_to_counts[variable][value] += weight
  # normalize
  variable_to_prob = normalize(variables_to_parents, variable_to_counts)
  return variable_to_prob

In [63]:
def EM_learning(bayes_data, variable_to_probability, variable_to_parents, epsilon):
  greaterThanEpsilon = True
  iteration = 0
  old_var_to_prob = variable_to_probability
  while greaterThanEpsilon and iteration < 50:
    assigment_to_norm_prob = e_step(bayes_data, old_var_to_prob, variable_to_parents)
    updated_var_to_prob = m_step(assigment_to_norm_prob, variable_to_parents, .0001, bayes_data)
    # determine if we've converged
    # for us, convergence happens if all of the unknown thetas change by less than epsilon
    # right now, unknown thetas are probabilities of time_delay
    greaterThanEpsilon = False
    for value in old_var_to_prob['time_delay']:
      diff = abs(old_var_to_prob['time_delay'][value] - updated_var_to_prob['time_delay'][value])
      if diff > epsilon:
        greaterThanEpsilon = True
        break
    iteration += 1
    print('iteration: {} diff: {}'.format(iteration, diff))
    print(updated_var_to_prob['time_delay'][(1, '7', '1', '13')])
    old_var_to_prob = updated_var_to_prob
  return updated_var_to_prob

In [64]:
final_var_to_prob = EM_learning(bayes_data, variable_to_probability, variables_to_parents, 0.0015)

iteration: 1 diff: 0.00302125381271
0.203021253813
iteration: 2 diff: 0.00166122055899
0.204682474372
iteration: 3 diff: 0.00845405650283
0.204825931815
iteration: 4 diff: 0.00983197021073
0.203550195371
iteration: 5 diff: 0.00256562123833
0.200984574133
iteration: 6 diff: 0.00369390636499
0.197290667768
iteration: 7 diff: 0.00463614003272
0.192654527735
iteration: 8 diff: 0.00537967911829
0.187274848617
iteration: 9 diff: 0.0059235622002
0.181351286416
iteration: 10 diff: 0.0062758017821
0.175075484634
iteration: 11 diff: 0.00644999627553
0.168625488359
iteration: 12 diff: 0.00646298358637
0.162162504772
iteration: 13 diff: 0.00633429377429
0.155828210998
iteration: 14 diff: 0.00608670488032
0.149741506118
iteration: 15 diff: 0.00574654271574
0.143994963402
iteration: 16 diff: 0.00534279454233
0.13865216886
iteration: 17 diff: 0.00490495895181
0.133747209908
iteration: 18 diff: 0.0044602244765
0.129286985431
iteration: 19 diff: 0.00403084389037
0.125256141541
iteration: 20 diff: 0.003

In [65]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [66]:
flights_data_test = bucket.item('model-test.csv')
uri = flights_data_test.uri
%gcs read --object $uri --variable data
test_data = pd.read_csv(BytesIO(data))
true_labels = test_data[['ARRIVAL_DELAY']].values

In [116]:
# test 1
guessed_labels = []
for index, row in test_data.iterrows():
  max_prob = 0
  max_val = -1
  for arrival_delay_val in range(3): # arrival delay can be 0, 1 or 2
    for time_delay_val in range(NUM_TIME_DELAY_BUCKETS):
      parent_list = variables_to_parents['ARRIVAL_DELAY']
      value = str(float(arrival_delay_val))
      value = [value]
      for parent in parent_list:
        if parent == 'time_delay':
          value.append(time_delay_val)
        else:
          value.append(str(row[parent]))
      value = tuple(value)
      prob = final_var_to_prob['ARRIVAL_DELAY'].get(value, .00000000000001)
      if prob > max_prob:
        max_prob = prob
        max_val = arrival_delay_val  
  guessed_labels.append(max_val)

0.307805919526
('0.0', 0, 'OO', 'CVG')
0.586935925353
('0.0', 1, 'OO', 'CVG')
0.125829350326
('0.0', 2, 'OO', 'CVG')
0.188332394033
('0.0', 3, 'OO', 'CVG')
0.208607427004
('0.0', 4, 'OO', 'CVG')
0.334226052116
('1.0', 0, 'OO', 'CVG')
0.10064125512
('1.0', 1, 'OO', 'CVG')
0.493329487873
('1.0', 2, 'OO', 'CVG')
0.673225806132
('1.0', 3, 'OO', 'CVG')
0.333439497508
('1.0', 4, 'OO', 'CVG')
0.357968028357
('2.0', 0, 'OO', 'CVG')
0.312422819527
('2.0', 1, 'OO', 'CVG')
0.380841161801
('2.0', 2, 'OO', 'CVG')
0.138441799835
('2.0', 3, 'OO', 'CVG')
0.457953075488
('2.0', 4, 'OO', 'CVG')


In [112]:
# test 2
guessed_labels = []
for index, row in test_data.iterrows():
  prob = 0
  max_val = 0
  for time_delay_val in range(NUM_TIME_DELAY_BUCKETS):
    parent_list = variables_to_parents['time_delay']
    value = time_delay_val
    value = [value]
    for parent in parent_list:
      value.append(str(row[parent]))
    value = tuple(value)
    prob = final_var_to_prob['time_delay'].get(value, .00000000000001)
    if prob > max_prob:
      max_prob = prob
      max_val = arrival_delay_val    
  time_delay_val = max_val
  max_prob = 0
  max_val = -1
  for arrival_delay_val in range(3): # arrival delay can be 0, 1 or 2
    parent_list = variables_to_parents['ARRIVAL_DELAY']
    value = str(float(arrival_delay_val))
    value = [value]
    for parent in parent_list:
      if parent == 'time_delay':
        value.append(time_delay_val)
      else:
        value.append(str(row[parent]))
    value = tuple(value)
    prob = final_var_to_prob['ARRIVAL_DELAY'].get(value, .00000000000001)
    if prob > max_prob:
      max_prob = prob
      max_val = arrival_delay_val    
  guessed_labels.append(max_val)

In [113]:
lr_test_acc = metrics.accuracy_score(true_labels, guessed_labels)
pred_y_test = [1 if guessed_labels[i] > 1 else 0 for i in range(len(guessed_labels))]
test_y_binarized = [1 if true_labels[i][0] > 1 else 0 for i in range(len(true_labels))]
print (len(test_y_binarized))
print ('baseline test acc: ' + str(lr_test_acc))
print ('baseline precision: ' + str(metrics.precision_score(test_y_binarized, pred_y_test)))

17700
baseline test acc: 0.3463276836158192
baseline precision: 0.3426955702167766


In [114]:
print(true_labels[:5])
print(guessed_labels[:5])

[[0.]
 [1.]
 [1.]
 [0.]
 [2.]]
[2, 0, 1, 0, 2]


In [115]:
import cPickle as pickle

json = pickle.dumps(final_var_to_prob)
bucket.item('hidden_time_var.txt').write_to(json,'text/txt')